In [5]:

from langchain.chat_models import ChatOpenAI



chat = ChatOpenAI(
    temperature=0.1
)


b = chat.predict('How many planets are there?')

b

'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

In [6]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

# system message는 AI에 일종의 기본 설정, 기본 값, 기본 context 설정을 한다.(AI의 설정) 즉, ai의 rp를 설정했다 보면됨
# ai message는 가상의 대화화
# human message는 실제 대화 메시지

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italian"),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Mexico and Thailand. Alos what is your name?")
]

chat.predict_messages(messages)

AIMessage(content='Ciao! La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Mi chiamo Paolo, sono un esperto di geografia. Posso aiutarti con altre domande?')